<a href="https://www.kaggle.com/code/anandsiva/gpt-research-tool-gemini-rag?scriptVersionId=167430913" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://colab.research.google.com/github/pvanand07/GPT-Research-tool/blob/master/gpt_research_main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-Research-Tool | Market Research on Canoo ev
GPT Research tool for market research, competition analysis, etc. Built using Gemini API, Beautiful Soup and FastEmbed | Inspired from [GPT Researcher by assafelovic](https://github.com/assafelovic/gpt-researcher)

**Author: Anand Siva**| [Github](https://github.com/pvanand07/GPT-Research-tool) | [Linkedin](https://www.linkedin.com/in/anandsivapv/) | [Portfolio](https://anand-analyst.carrd.co/)


## Summary
This solution involves creating research questions from user query, gathering website data through Google Custom Search API and web scraping, processing data using NLP, using vector embeddings and semantic search to obtain relevant information and generating insights with generative AI.



### API's required
- google_search_api || Get your Google Search API key from [here](https://developers.google.com/custom-search/v1/overview) (free) (LIMIT:100 queries/day)
- cse_id || Obtain your Custom Search Engine ID from [here](https://programmablesearchengine.google.com/controlpanel/all) (free)
- GEMINI_API_KEY || Get your GEMINI API key from [here](https://aistudio.google.com/app/apikey) (free) (LIMIT:100 queries/hour)

In [20]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from IPython.display import HTML
HTML('<iframe width="1280" height="720" src="https://www.youtube.com/embed/m77uInzXNMI" title="⚡GPT-Researcher-Gemini-RAG⚡" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>')

<div>

</div>

## Steps Involved in Creating the Solution

### Install and Import Libraries:
Installing necessary Python packages like `google-generativeai` for accessing Google's generative AI models and `qdrant-client` for leveraging Qdrant's capabilities for fast embeddings on CPU. Essential libraries like `nltk`, `requests`, `bs4` (BeautifulSoup), and others are imported for text processing and web scraping.

### Define Research Questions:
The project aims to analyze Canoo EV's industry, competitors, market trends, and financial performance through a set of predefined research questions.

### Data Gathering:
- Utilizing Google's Custom Search API to gather data based on generated search queries relevant to the research questions.
- Scraping web content using BeautifulSoup for detailed analysis.

### Data Processing:
- Tokenizing and cleaning the scraped data using `nltk`.
- Organizing data into a structured format (dictionary) for analysis.

### Analysis and Report Generation:
- Employing generative AI (Gemini model from Google Generative AI) to interpret the gathered data and generate insights.
- Writing the insights into a markdown file as a comprehensive report.

## Techniques and Libraries Used

- **Natural Language Processing (NLP):** Used for generating multiple research queries for the user question, processing text data, and summarizing content. Libraries like `nltk` for tokenization and `google.generativeai` for generating content based on questions.
- **APIs for Data Gathering:** Google's Custom Search API for fetching relevant online links for research queries.
- **Web Scraping:** Utilizing `requests` and `BeautifulSoup` to extract information from web pages.
- **Concurrent Processing:** Using `ThreadPoolExecutor` for efficient data gathering and processing.
- **Qdrant Client:** For leveraging fast embeddings using quantization and semantic search.
- **sentence-transformers/all-MiniLM-L6-v2:** Maps sentences & paragraphs to a 384 dimensional dense vector space.

## 1. Install Required Libraries

In [1]:
# To Measure Notebook runtime
import time
start_time = time.time()

In [2]:
# Requirements
# qdrant utilizes quantized models for fast inference on cpu
!pip install qdrant-client[fastembed] -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.3 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.0 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<4.0.0dev,>=3.12.0, but you have protobuf 4.25.3 which is incompatible.
google-cloud-bigtable 1.7.3 requires protobuf<4.0.0dev, but you have protobuf 4.25.3 which is incompatible.
google-cloud-datastore 

## 2. Define Imports and Constants

In [3]:
research_questions = [
   "What is a GM cruise.",
"What Technology GM Cruise Use",
"What are the challenges GM Cruise is facing",
"What is the future of GM Cruise."

]

In [4]:
import nltk
nltk.download('punkt')
import requests
from IPython.display import display
from IPython.display import Markdown
import pathlib
import textwrap
import google.generativeai as genai
from googleapiclient.discovery import build
from nltk.tokenize import sent_tokenize
import ast
import json
from qdrant_client import QdrantClient
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai

from kaggle_secrets import UserSecretsClient

my_api_key = UserSecretsClient().get_secret('google_search_api') # Get your Google Search API key from here (free) (LIMIT:100 queries/day): https://developers.google.com/custom-search/v1/overview
my_cse_id = UserSecretsClient().get_secret('cse_id') # Obtain your Custom Search Engine ID from here (free): https://programmablesearchengine.google.com/controlpanel/all
GEMINI_API_KEY = UserSecretsClient().get_secret('gemini_api_key') # Get your GEMINI API key from here (free) (LIMIT:100 queries/hour) https://aistudio.google.com/app/apikey

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 3. Generate Research Questions

In [5]:
def google_search(search_term, api_key, cse_id, **kwargs):
    """
    Perform a Google custom search for a given search term using a specific API key and Custom Search Engine ID.

    Args:
        search_term (str): The query or term to search for.
        api_key (str): The API key to authenticate with Google's Custom Search API.
        cse_id (str): The Custom Search Engine ID to specify which search engine to use.
        **kwargs: Arbitrary keyword arguments that are passed to the custom search engine list method.

    Returns:
        dict: The search result in a dictionary format.
    """
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

def to_markdown(text):
    """
    Convert text to Markdown format.
    Args: text (str): The text to convert to Markdown.
    Returns: str: The text converted to Markdown format.
    """
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def ask_gemini(prompt, GEMINI_API_KEY):
    """
    Generate a response for the given prompt using the Gemini AI model.
    Args:
        prompt (str): The input prompt to generate a response for.
        GEMINI_API_KEY (str): The API key for accessing the Gemini AI service.
    Returns:
        str: The response generated by the Gemini AI model.
    """
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    display(to_markdown(response.text))

    return response.text

In [6]:
def generate_search_queries_prompt(question, max_iterations=6):
    """ Generates the search queries prompt for the given question.
    Args: question (str): The question to generate the search queries prompt for
    Returns: str: The search queries prompt for the given question
    """

    return 'Follow the instructions'\
           f'1. Write {max_iterations} google search queries to search online that form an objective opinion from the following: "{question}"' \
           f'2. output only the list of search queries in the format: ["query 1", "query 2", ...].'


def generate_report_prompt(question, context, report_format="apa", total_words=1000):
    """ Generates the report prompt for the given question and research summary.
    Args: question (str): The question to generate the report prompt for
            research_summary (str): The research summary to generate the report prompt for
    Returns: str: The report prompt for the given question and research summary
    """

    return f'Using the Information: """{context}"""\n\n' \
           f'answer the following' \
           f'query : "{question}" in a detailed report --' \
           f"with facts and numbers if available and a minimum of {total_words} words.\n" \
           f"You MUST write the report in {report_format} format.\n "

In [7]:
def generate_search_queries(research_questions):
    """
    Generate search queries for a list of research questions using the Gemini AI model.

    Args:
        research_questions (list of str): A list of research questions to generate search queries for.

    Returns:
        dict: A dictionary mapping each research question to its corresponding generated search queries.
    """
    research_queries_dict ={}
    for question in research_questions:
      question_prompt = generate_search_queries_prompt(question)
      research_queries_dict[question] = ask_gemini(question_prompt,GEMINI_API_KEY)

    return research_queries_dict


def generate_research_query_list(research_queries_dict):
    """
    Generate a list of research queries from the output of the Gemini AI model.

    Args:
        research_queries_dict (dict): A dictionary mapping research questions to their corresponding Gemini-generated queries.

    Returns:
        dict: A dictionary mapping each research question to a list of generated research queries.
    """
    # Convert Gemini output from string to list
    research_queries_dict_list = {}
    for question, research_queries in research_queries_dict.items():
      if research_queries and research_queries.strip():  # Check if the input is not null or empty
          try:
              research_queries_dict_list[question] = ast.literal_eval(research_queries)

          except SyntaxError or ValueError:
              # If ast.literal_eval() fails due to syntax error, tokenize the string into sentences using NLTK
              research_queries_dict_list[question] = sent_tokenize(research_queries)
              if len(research_queries_dict_list[question]) == 1:
                research_queries_dict_list[question] = research_queries.split('\n')
      else:
          print("Input ask_gemini(question)-->research_queries is null or empty. Passing...")

    return research_queries_dict_list

research_queries_dict = generate_search_queries(research_questions)
research_queries_dict_list = generate_research_query_list(research_queries_dict)

> ["What is General Motors Cruise?", "Cruise self-driving technology", "Cruise autonomous vehicles", "Cruise history and development", "Cruise current operations", "Cruise future plans"]

> ["GM Cruise Technology Overview", "GM Cruise Sensor Suite", "GM Cruise Hardware Architecture", "GM Cruise Software Stack", "GM Cruise Lidar System", "GM Cruise Mapping Technology"]

> ["Challenges faced by GM Cruise", "GM Cruise obstacles and hurdles", "Difficulties encountered by GM Cruise", "Problems and setbacks for GM Cruise", "Barriers to GM Cruise's progress", "Issues limiting GM Cruise's success"]

> ["GM Cruise future projections", "GM Cruise market analysis", "Industry experts' opinions on GM Cruise", "GM Cruise financial outlook", "Potential challenges facing GM Cruise", "GM Cruise's impact on the transportation industry"]

## 4. Extracting relevant links using Google Search

In [8]:
def extract_data_from_google_search_results(research_queries_dict_list):
    """
    Extract data from Google search results based on the provided research query list.

    Args:
        research_queries_dict_list (dict): A dictionary mapping research questions to a list of generated research queries.

    Returns:
        dict: A dictionary containing Google search results, where keys are the search queries and values are the corresponding search results.
    """
    # Using the search queries to perform google search and extract links
    research_links_google = {}

    for question, research_queries_list in research_queries_dict_list.items():
      for research_query in research_queries_list:
        research_links_google[research_query] = google_search(research_query, my_api_key, my_cse_id, num=2)

    # Writing google search results dictionary to a JSON file
    with open("research_links_google.json", 'w') as json_file:
        json.dump(research_links_google, json_file)

    print(f"Dictionary saved as JSON file: research_links_google.json")

    return research_links_google

research_links_google = extract_data_from_google_search_results(research_queries_dict_list)

Dictionary saved as JSON file: research_links_google.json


In [9]:
def extract_links(research_links_google):
    """
    Extract links from the Google search results stored in a dictionary.

    Args:
        research_links_google (dict): A dictionary containing Google search results.

    Returns:
        list: A list of URLs extracted from the search results.
    """
    # Load the search results from the JSON file
    # File path to the JSON file
    file_path = "/kaggle/working/research_links_google.json"

    # Loading JSON file as dictionary
    with open(file_path, 'r') as json_file:
        research_links_google1 = json.load(json_file)


    topics_dict_list = [research_links_google[k] for k, v in research_links_google.items()] # 24 Topics x 2 URLS

    links_list = []
    for topic_dict in topics_dict_list:
      try:
        for item in topic_dict['items']:
          links_list.append(item['link'])
      except:
        pass

    return links_list

links_list = extract_links(research_links_google)

## 5. Web Scraping

In [10]:
# Scraper class from: https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/scraper/scraper.py

class Scraper:
    """
    Scraper class to extract the content from web page links.
    """
    def __init__(self, urls, user_agent):
        """
        Initialize the Scraper class.
        Args:
            urls (list): List of URLs to scrape.
            user_agent (str): User agent string for the HTTP header to simulate a browser request.
        """
        self.urls = urls
        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": user_agent
        })

    def run(self):
        """
        Extracts the content from the provided URLs.
        Returns:
            list: A list of dictionaries with URL and its corresponding raw content.
        """
        partial_extract = partial(self.extract_data_from_link, session=self.session)
        with ThreadPoolExecutor(max_workers=20) as executor:
            contents = executor.map(partial_extract, self.urls)
        res = [content for content in contents if content['raw_content']]
        return res

    def extract_data_from_link(self, link, session):
        """
        Extracts the data from a single web page link.
        Args:
            link (str): URL to scrape.
            session (requests.Session): Session object for making requests.
        Returns:
            dict: A dictionary with URL and its raw content.
        """
        try:
            response = session.get(link, timeout=4)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'lxml', from_encoding=response.encoding)
                raw_content = self.scrape_text_with_bs(soup)
                if len(raw_content) < 100:
                    return {'url': link, 'raw_content': None}
                return {'url': link, 'raw_content': raw_content}
            return {'url': link, 'raw_content': None}
        except Exception as e:
            return {'url': link, 'raw_content': None}

    def scrape_text_with_bs(self, soup):
        """
        Extracts and cleans text content from a BeautifulSoup object.
        Args:
            soup (BeautifulSoup): BeautifulSoup object of a web page.
        Returns:
            str: Cleaned text content from the web page.
        """
        for script_or_style in soup(["script", "style"]):
            script_or_style.extract()  # Remove script and style elements

        text = soup.get_text(separator="\n")
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        content = "\n".join(chunk for chunk in chunks if chunk)
        return content


In [11]:
def extract_and_store_data(links_list, user_agent, output_file):
    """
    Extracts data from the provided URLs and stores the data as a JSON file.

    Args:
        research_links_google_list (dict): A dictionary with keys as topics and values as lists of URLs.
        user_agent (str): User agent string for HTTP requests.
        output_file (str): Path to the output JSON file.
    """

    # Initialize the Scraper with URLs and a user agent
    scraper = Scraper(links_list, user_agent)
    # Extract data
    data = scraper.run()

    data_dict = {}

    for item in data:
      data_dict[item['url']] = item['raw_content']
    data_dict

    # Write the data to a JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data_dict, f, ensure_ascii=False, indent=4)

# Usage
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
output_file = 'extracted_data.json'  # Specify the path to the output file

# Call the function with the provided research links, user agent, and output file path
extract_and_store_data(links_list, user_agent, output_file)


In [12]:
# File path to the JSON file
file_path = "/kaggle/working/extracted_data.json"

# Loading JSON file as dictionary
with open(file_path, 'r') as json_file:
    extracted_data = json.load(json_file)
len(extracted_data)

30

## 6. Creating Vector Embeddings

In [13]:
%%time
def add_to_qdrant(extracted_data):
    """
    Add data to the Qdrant vector similarity search engine.

    Args:
        extracted_data (dict): A dictionary containing extracted data where keys are URLs and values are text data.

    Returns:
        QdrantClient: An instance of the Qdrant client with the data added.
    """

    # Initialize the Qdrant client with in-memory storage for demonstration purposes
    client = QdrantClient(":memory:")  # Adjust the host and port if necessary
    client.set_model("sentence-transformers/all-MiniLM-L6-v2")

    # Assuming `extracted_data` is a pre-defined dictionary with your data
    documents = []
    metadatas = []
    ids = []

    # Prepare your documents and metadata
    for url, data in extracted_data.items():
      sentences = data.split('\n')

      # Here you might need to adjust how you handle IDs to ensure they are unique integers
      # Qdrant currently requires numeric IDs
      for i, sentence in enumerate(sentences):
          documents.append(sentence)
          metadatas.append({"url": url+str(i)})
          ids.append(hash(f"{url}_{i}"))  # Example way to generate a numeric ID

    # Add documents to Qdrant
    client.add(
        collection_name="your_collection_name",
        documents=documents,
        metadata=metadatas,
        ids=ids
    )
    return client

client = add_to_qdrant(extracted_data)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

CPU times: user 5min 24s, sys: 2.03 s, total: 5min 26s
Wall time: 2min 44s


## 7. Vector Similarity Search using Qdrant Client

In [14]:
def query_index(client,collection_name, query_text, limit=100):
    """ Query the index with the given client and query text.
    """

    search_result = client.query(
    collection_name=collection_name,
    query_text=query_text, limit= limit
    )
    context = "".join(set(["".join(result.metadata['document']) for result in search_result]))

    return context

## 8. Generate a detailed Research Report using GEMINI

In [15]:
%%time
def create_report(client, research_questions):
    """
    Create a research report by generating responses to a list of research questions.

    Args:
        client (QdrantClient): An instance of the Qdrant client.
        research_questions (list of str): A list of research questions to create the report for.

    Returns:
        list: A list of responses generated for each research question.
    """
    response = []
    display(to_markdown("# Your Research Report"))
    for question in research_questions:
      context = query_index(client, "your_collection_name", question)
      prompt = generate_report_prompt(question, context[:31000*4])
      response.append(ask_gemini(prompt, GEMINI_API_KEY))
      display(to_markdown("# USER: "+question))
        
    return response

response = create_report(client, research_questions)

> # Your Research Report

> **General Motors Cruise**
> 
> **Introduction**
> 
> General Motors Cruise is a company that develops autonomous vehicle technology. The company was founded in 2013 by Kyle Vogt, Daniel Kan, and Talmon Hessel, and is headquartered in San Francisco, California. Cruise is majority-owned by General Motors (GM), which acquired the company in 2016.
> 
> **Technology**
> 
> Cruise's autonomous vehicle technology is based on a combination of sensors, cameras, and radar. The sensors collect data about the vehicle's surroundings, which is then processed by the cameras and radar to create a 3D map of the environment. The map is used by the vehicle's software to plan a safe path to its destination.
> 
> Cruise's autonomous vehicle technology has been tested extensively on public roads. In 2017, the company launched a pilot program in San Francisco that allowed members of the public to ride in self-driving cars. The pilot program was successful, and Cruise has since expanded its testing to other cities, including Phoenix, Arizona; Austin, Texas; and Detroit, Michigan.
> 
> **Vehicles**
> 
> Cruise's autonomous vehicle technology is currently being used in two different vehicles: the Chevrolet Bolt EV and the Origin. The Bolt EV is a compact electric car that is used for Cruise's ride-hailing service. The Origin is a purpose-built autonomous vehicle that is being developed for use in commercial fleets.
> 
> The Origin is a fully autonomous vehicle that does not have a steering wheel or pedals. The vehicle is designed to be safe, comfortable, and efficient. The Origin is also designed to be accessible to people with disabilities.
> 
> **Business Model**
> 
> Cruise's business model is based on providing autonomous vehicle ride-hailing services. The company plans to launch its ride-hailing service in San Francisco in 2024. Cruise also plans to offer its autonomous vehicle technology to other companies, such as fleet operators and delivery companies.
> 
> **Funding**
> 
> Cruise has raised a total of $14 billion in funding from investors, including GM, Honda, SoftBank Vision Fund, Walmart, and Microsoft. The company's most recent funding round was a $5 billion investment from GM in 2023.
> 
> **Challenges**
> 
> Cruise faces a number of challenges, including regulatory hurdles, technical challenges, and public acceptance.
> 
> **Regulatory hurdles**
> 
> Cruise must obtain approval from regulators before it can deploy its autonomous vehicles on public roads. The company is currently working with regulators in California, Arizona, Texas, and Michigan to obtain the necessary approvals.
> 
> **Technical challenges**
> 
> Cruise's autonomous vehicle technology is still under development. The company is working to improve the safety and reliability of its technology.
> 
> **Public acceptance**
> 
> Cruise must convince the public that its autonomous vehicles are safe and reliable. The company is working to build trust with the public through its pilot programs and public relations efforts.
> 
> **Conclusion**
> 
> Cruise is a leading company in the development of autonomous vehicle technology. The company has raised a significant amount of funding and is working with regulators to obtain the necessary approvals to deploy its vehicles on public roads. Cruise faces a number of challenges, but the company is well-positioned to overcome these challenges and become a leader in the autonomous vehicle industry.

> # USER: What is a GM cruise.

> **Title**: GM Cruise Technology
> 
> **Abstract:**
> 
> General Motors' Cruise is a self-driving car company that is developing autonomous vehicles for commercial use. Cruise's technology stack includes sensors, software, and hardware that enable its vehicles to navigate roads, avoid obstacles, and interact with other road users. This report provides a detailed overview of Cruise's technology, including its sensor suite, computer vision system, and machine learning algorithms.
> 
> **Sensors**
> 
> Cruise's vehicles are equipped with a suite of sensors that provide a 360-degree view of the surrounding environment. These sensors include:
> 
> * **Cameras:** Cruise's vehicles are equipped with multiple cameras that provide a wide field of view. The cameras are used to detect objects, track their movement, and identify traffic signs and signals.
> * **RADAR:** Cruise's vehicles are also equipped with RADAR sensors that emit radio waves to detect objects in the surrounding environment. RADAR is used to measure the distance and speed of objects, and to track their movement.
> * **LIDAR:** Cruise's vehicles are equipped with LIDAR sensors that emit laser beams to create a 3D map of the surrounding environment. LIDAR is used to detect objects, identify their shape and size, and to track their movement.
> * **Ultrasonic sensors:** Cruise's vehicles are also equipped with ultrasonic sensors that emit high-frequency sound waves to detect objects in the surrounding environment. Ultrasonic sensors are used to detect objects that are close to the vehicle, such as other vehicles, pedestrians, and cyclists.
> 
> **Software**
> 
> Cruise's software stack is responsible for processing the data from the sensor suite and making decisions about how to navigate the vehicle. The software stack includes:
> 
> * **Computer vision:** Cruise's computer vision system uses machine learning algorithms to identify objects in the surrounding environment. The computer vision system is used to detect vehicles, pedestrians, cyclists, traffic signs, and signals.
> * **Path planning:** Cruise's path planning system uses machine learning algorithms to generate a safe and efficient path for the vehicle to follow. The path planning system takes into account the vehicle's current location, the surrounding environment, and the vehicle's destination.
> * **Control:** Cruise's control system uses machine learning algorithms to control the vehicle's steering, braking, and acceleration. The control system is responsible for keeping the vehicle on track, avoiding obstacles, and maintaining a safe speed.
> 
> **Hardware**
> 
> Cruise's vehicles are equipped with a variety of hardware components that support the software stack. These hardware components include:
> 
> * **Compute:** Cruise's vehicles are equipped with high-performance computers that are responsible for processing the data from the sensor suite and running the software stack.
> * **Storage:** Cruise's vehicles are equipped with large storage devices that are used to store data from the sensor suite and the software stack.
> * **Networking:** Cruise's vehicles are equipped with networking devices that allow them to communicate with other vehicles, infrastructure, and the cloud.
> 
> **Performance**
> 
> Cruise's technology has been extensively tested in real-world conditions. In 2019, Cruise's vehicles completed a cross-country drive from San Francisco to New York City without any human intervention. The drive covered over 3,300 miles and included a variety of driving conditions, including highways, city streets, and rural roads.
> 
> **Safety**
> 
> Safety is a top priority for Cruise. Cruise's vehicles are designed with a number of safety features, including:
> 
> * **Multiple redundancies:** Cruise's vehicles are equipped with multiple redundant systems for critical functions, such as steering, braking, and acceleration. This redundancy helps to ensure that the vehicle will continue to operate safely in the event of a failure of one of the systems.
> * **Fail-safe design:** Cruise's vehicles are designed to fail in a safe manner. In the event of a system failure, the vehicle will automatically stop and activate its hazard lights.
> * **Human oversight:** Cruise's vehicles are designed to be overseen by a human operator. The human operator can intervene at any time to take control of the vehicle.
> 
> **Conclusion**
> 
> Cruise's technology is at the forefront of the autonomous vehicle industry. Cruise's vehicles are equipped with a suite of sensors, software, and hardware that enable them to navigate roads, avoid obstacles, and interact with other road users. Cruise's technology has been extensively tested in real-world conditions and has been shown to be safe and reliable.
> 
> **References**
> 
> * [Cruise website](https://www.cruise.com/)
> * [GM website](https://www.gm.com/)
> * [NHTSA website](https://www.nhtsa.gov/)

> # USER: What Technology GM Cruise Use

> **Title:** Challenges Faced by General Motors' Cruise Division
> 
> **Abstract:**
> 
> General Motors' (GM) Cruise division has encountered a range of challenges since its inception in 2016. These challenges include regulatory and safety issues, financial difficulties, and operational setbacks. This report provides a detailed overview of the challenges faced by Cruise, with an emphasis on facts and figures.
> 
> **Introduction:**
> 
> Cruise is a self-driving car company majority-owned by GM. The company was founded in 2016 with the goal of developing and deploying autonomous vehicles for commercial use. However, Cruise has faced a number of challenges that have hindered its progress.
> 
> **Regulatory and Safety Issues:**
> 
> One of the biggest challenges facing Cruise is the regulatory landscape surrounding autonomous vehicles. Cruise has been involved in several accidents, including a fatal crash in 2022, which have raised concerns about the safety of its technology. As a result, Cruise has been forced to suspend operations in several cities and has come under increased scrutiny from regulators.
> 
> **Financial Difficulties:**
> 
> Cruise has also faced financial difficulties. The company has lost over $8 billion since 2016, and it is unclear when it will become profitable. GM has been forced to provide Cruise with significant financial support, which has strained the automaker's own finances.
> 
> **Operational Setbacks:**
> 
> In addition to regulatory and financial challenges, Cruise has also faced operational setbacks. The company has struggled to scale up its operations and has had difficulty meeting its production targets. Cruise has also been plagued by technical problems, which have delayed the deployment of its self-driving vehicles.
> 
> **Specific Examples:**
> 
> Here are some specific examples of the challenges faced by Cruise:
> 
> * In October 2022, a Cruise self-driving vehicle struck and killed a pedestrian in San Francisco. The accident raised concerns about the safety of Cruise's technology and led to the suspension of its operations in the city.
> * In 2023, the California Department of Motor Vehicles (DMV) suspended Cruise's operating permit after an investigation found that the company had failed to adequately address safety concerns. The DMV also found that Cruise had misled regulators about its technology.
> * In 2023, Cruise announced that it would lay off 20% of its workforce, or about 900 employees. The layoffs were part of a cost-cutting measure designed to improve the company's financial performance.
> * In 2024, Cruise announced that it would delay the launch of its commercial robotaxi service until 2025. The delay was due to a number of factors, including regulatory hurdles and technical challenges.
> 
> **Conclusion:**
> 
> The challenges facing GM's Cruise division are significant. The company has faced regulatory scrutiny, financial difficulties, and operational setbacks. It is unclear when Cruise will be able to overcome these challenges and become a profitable business. However, GM remains committed to Cruise and believes that the company has the potential to be a major player in the autonomous vehicle market.

> # USER: What are the challenges GM Cruise is facing

> **The Future of GM Cruise**
> 
> **Background**
> 
> Cruise, a self-driving car company majority-owned by General Motors (GM), has faced a series of setbacks in recent years, including regulatory investigations and safety concerns. In October 2023, the California Department of Motor Vehicles (DMV) suspended Cruise's operating permit following an accident in which a pedestrian was dragged 20 feet by a Cruise self-driving vehicle. Cruise is also under investigation by the U.S. Department of Justice and the U.S. Securities and Exchange Commission.
> 
> **Challenges**
> 
> Cruise has faced a number of challenges in recent years, including:
> 
> * **Regulatory hurdles:** Cruise has faced regulatory hurdles in a number of states, including California and Arizona. The company has been required to obtain permits to operate its self-driving vehicles, and it has faced scrutiny from regulators over its safety record.
> * **Safety concerns:** Cruise has been involved in a number of accidents, including the fatal crash of a pedestrian in San Francisco in 2021. These accidents have raised concerns about the safety of Cruise's self-driving technology.
> * **Financial losses:** Cruise has been a financial drain on GM, with losses over $700 million in the third quarter of 2023 and more than $8 billion since 2016. These losses have been increasing, and they have put pressure on GM to reconsider its investment in Cruise.
> 
> **GM's Response**
> 
> GM has responded to Cruise's challenges by taking a number of steps, including:
> 
> * **Increased investment:** GM has announced that it will invest an additional $1.1 billion in Cruise. This investment will be used to fund Cruise's development of new self-driving technology and to expand its operations.
> * **New leadership:** GM has appointed a new CEO for Cruise, Kyle Vogt. Vogt is a former engineer at Google, and he has experience in developing self-driving technology.
> * **Safety improvements:** Cruise has made a number of safety improvements to its self-driving technology, including adding new sensors and cameras to its vehicles. The company has also updated its software to address the underlying issues that led to the fatal crash in San Francisco.
> 
> **Outlook**
> 
> The future of GM Cruise is uncertain. The company faces a number of challenges, including regulatory hurdles, safety concerns, and financial losses. However, GM has taken a number of steps to address these challenges, and it is committed to Cruise's long-term success.
> 
> **Conclusion**
> 
> Cruise is a high-risk, high-reward investment for GM. The company has the potential to revolutionize the transportation industry, but it also faces a number of challenges. GM is committed to supporting Cruise, and it is likely that the company will continue to invest in Cruise's development. However, it is unclear whether Cruise will be able to overcome its challenges and achieve commercial success.
> 
> **References**
> 
> * "GM's Cruise gets $2.25B from SoftBank's Vision Fund, $1.1B from GM". Cruise LLC. October 2023.
> * "GM Announces Additional Investment in Cruise". GM. December 2023.
> * "Cruise CEO quits after alleged cover-up". The Verge. January 2024.
> * "Probe into GM's Cruise finds poor leadership, culture issues at center of accident response". The New York Times. January 2024.
> * "GM's Cruise Origin Is an Autonomous Vehicle From the Future". GM. 2023.
> * "GM's Cruise laying off 900 employees, or 24% of its workforce: Read the memo here". TechCrunch. January 2024.
> * "GM had poured billions into Cruise as the company emphasized scaling up at an unprecedented pace". The Wall Street Journal. January 2024.
> * "Cruise, which GM acquired in 2016, was considered to be among the leaders in autonomous vehicles". Reuters. January 2024.

> # USER: What is the future of GM Cruise.

CPU times: user 1.26 s, sys: 860 ms, total: 2.12 s
Wall time: 56.2 s


In [16]:
with open("generated_report.md", 'w') as report:
  report.write("".join(response))

In [17]:
with open("generated_report.md", 'w') as report:
  report.write("".join(response))

end_time = time.time()
runtime = end_time - start_time

print(f"Notebook Runtime is {int(runtime // 60)} minutes and {int(runtime % 60)} seconds")

Notebook Runtime is 4 minutes and 55 seconds


# EXAMPLE 2 - CANOO EV - FINAL REPORT IN MARKDOWN FORMAT

In [18]:
# These are the research questions that are fed into the above function and following report is obtained
research_questions = [
    "Identify the industry in which Canoo ev operates, along with its size, growth rate, trends, and key players.",
    "Analyze Canoo ev's main competitors, including their market share, products or services offered, pricing strategies, and marketing efforts.",
    "Identify key trends in the ev market, including changes in consumer behavior, technological advancements, and shifts in the competitive landscape.",
    "Gather information on Canoo ev's financial performance, including its revenue, profit margins, return on investment, and expense structure."
]

**Industry Overview**

**Industry:** Electric Vehicle (EV) Market

**Size:**

* The global EV market size was valued at USD 335.23 billion in 2023.
* It is projected to reach USD 1,318.09 billion by 2030, registering a CAGR of 24.1% from 2023 to 2030.

**Growth Rate:**

* The market is witnessing significant growth due to rising fuel prices, increasing environmental concerns, government initiatives promoting EV adoption, and technological advancements.

**Trends:**

* Shift towards electric mobility to reduce carbon emissions and combat climate change.
* Increasing demand for EVs from ride-hailing and delivery services.
* Growing popularity of EV subscription models and battery leasing options.
* Development of solid-state batteries and wireless charging technologies.

**Key Players:**

* BYD (China)
* Tesla (US)
* Volkswagen AG (Germany)
* SAIC Motors (China)
* Stellantis (Netherlands)
* Ford Motor Company (US)
* General Motors (US)
* Rivian (US)
* Lucid Motors (US)
* Nio (China)

**Canoo's Position in the Industry**

**About Canoo:**

* Canoo is an American electric vehicle company headquartered in Bentonville, Arkansas.
* It was founded in 2017 by Stefan Krause, Ulrich Kranz, and Richard Kim.
* Canoo designs, develops, and manufactures electric vehicles and mobility solutions.

**Market Share:**

* Canoo is a relatively new entrant in the EV market compared to established players like Tesla and Volkswagen.
* It has a small market share but is rapidly expanding its production capacity and product offerings.

**Growth Strategy:**

* Canoo is focused on developing innovative and affordable EVs for urban mobility.
* It emphasizes modular vehicle design, subscription-based ownership models, and advanced technology features.

**Key Partnerships:**

* Canoo has partnered with Hyundai to co-develop a new electric vehicle platform.
* It has also secured a deal with Walmart to supply 4,500 electric delivery vehicles.

**Financial Performance:**

* Canoo went public in 2020 through a merger with Hennessy Capital Acquisition Corp. IV.
* As of December 31, 2022, the company had total assets of $438.8 million and a net loss of $487.4 million.

**Challenges:**

* Canoo faces intense competition from established automakers and well-funded EV startups.
* The company has experienced production delays and supply chain issues.
* There is still uncertainty surrounding the adoption rate of EVs and the regulatory landscape for autonomous vehicles.

**Opportunities:**

* The EV market is rapidly growing, creating significant opportunities for new entrants.
* Canoo's modular vehicle design and subscription-based ownership models are differentiated offerings that could appeal to consumers.
* The company's partnerships with Hyundai and Walmart could provide a strong foundation for future growth.

**Conclusion**

The EV market is poised for significant growth in the coming years, driven by environmental concerns, government support, and technological advancements. Canoo is a promising player in this market with its innovative vehicle designs, subscription-based ownership models, and strategic partnerships. While the company faces challenges, it has the potential to capture a significant market share and become a major player in the EV industry.**Analysis of Canoo EV's Main Attributes**

**Company Overview**

Canoo is a Los Angeles-based electric vehicle (EV) company founded in 2017 by Stefan Krause, Ulrich Kranz, and Richard Kim. The company's mission is to bring EVs to everyone by developing affordable, sustainable, and stylish vehicles. Canoo has a team of experienced engineers and designers who have worked on some of the most iconic vehicles in the world, including the BMW i3 and the Faraday Future FF91.

**Market Share**

Canoo is a relatively new entrant to the EV market, but the company has quickly gained attention for its innovative design and affordable pricing. In 2022, Canoo had a market share of approximately 0.1% in the global EV market. The company's market share is expected to grow significantly in the coming years as it ramps up production of its vehicles.

**Products or Services Offered**

Canoo currently offers two models of EVs: the Lifestyle Vehicle and the Multi-Purpose Delivery Vehicle MPDV. The Lifestyle Vehicle is a compact SUV with a starting price of \$34,750. The MPDV is a commercial vehicle designed for businesses and fleets. It has a starting price of \$39,950.

Canoo's vehicles are unique in that they are built on a modular platform that allows for a variety of configurations. This platform can be used to create vehicles of different sizes and shapes, including sedans, coupes, and pickup trucks.

**Pricing Strategies**

Canoo's pricing strategies are designed to make its vehicles affordable for a wide range of consumers. The company offers a variety of financing options, including leases and loans. Canoo also offers a subscription service that allows customers to pay a monthly fee for access to a vehicle.

**Marketing Efforts**

Canoo has a strong marketing presence online and in print. The company has also partnered with a number of influencers and celebrities to promote its vehicles. Canoo's marketing efforts have been successful in generating interest in the company and its vehicles.

**Conclusion**

Canoo is a promising new entrant to the EV market. The company's innovative design, affordable pricing, and strong marketing efforts have helped it to quickly gain attention. Canoo is well-positioned to become a major player in the EV market in the coming years.

**Additional Information**

* Canoo has raised over $500 million in funding from investors, including Hyundai Motor Group and BlackRock.
* Canoo is planning to begin production of its vehicles in 2023.
* Canoo has plans to expand its product line to include a variety of different vehicle types, including sedans, coupes, and pickup trucks.
* Canoo is committed to sustainability and uses recycled materials in its vehicles.
* Canoo has a strong team of experienced engineers and designers.
* Canoo has a dedicated customer service team.
* Canoo offers a variety of financing options, including leases and loans.
* Canoo offers a subscription service that allows customers to pay a monthly fee for access to a vehicle.**Key Trends in the Electric Vehicle (EV) Market**

**Introduction**
The electric vehicle (EV) market is rapidly evolving as consumers become more environmentally conscious and governments implement policies to reduce carbon emissions. This report identifies key trends shaping the EV market, including changes in consumer behavior, technological advancements, and shifts in the competitive landscape.

**Changes in Consumer Behavior**

* **Increased Environmental Awareness:** Consumers are increasingly aware of the environmental impact of gasoline-powered vehicles and are seeking sustainable alternatives. EVs emit zero tailpipe emissions, making them a more environmentally friendly option.
* **Falling Costs:** EV prices have been steadily declining in recent years, making them more affordable for a wider range of consumers. Government incentives, such as tax credits and rebates, further reduce the cost of ownership.
* **Improved Performance:** EVs have made significant advancements in performance, including increased range, faster charging times, and improved acceleration. These improvements address common concerns about the practicality of EVs.
* **Changing Lifestyle:** The rise of ride-sharing and car-sharing services has reduced the need for personal vehicle ownership. EVs are well-suited for urban environments with short commutes and limited parking availability.

**Technological Advancements**

* **Battery Technology:** Battery technology is a critical factor in EV performance. Advances in battery chemistry and design have increased energy density, improved range, and reduced charging times.
* **Charging Infrastructure:** The expansion of EV charging infrastructure, including public charging stations and home charging units, makes it easier for consumers to charge their vehicles. Faster charging technologies further reduce the inconvenience of charging.
* **Autonomous Driving:** Autonomous driving technologies, such as lane keeping assist and adaptive cruise control, enhance the driving experience and safety of EVs.
* **Vehicle-to-Grid Technology:** EVs can store energy and release it back to the grid during peak demand periods. This technology provides benefits to both consumers and utilities.

**Shifts in the Competitive Landscape**

* **Traditional Automakers:** Traditional automakers, such as Ford, General Motors, and Volkswagen, are investing heavily in EV development to compete with Tesla and other EV startups.
* **EV Startups:** EV startups, such as Rivian, Lucid, and Fisker, are entering the market with innovative vehicle designs and features. They are challenging established automakers with their focus on sustainability and technology.
* **Technology Companies:** Technology companies, such as Apple and Google, are also developing EVs and investing in the EV ecosystem. They bring their expertise in software, artificial intelligence, and user experience.
* **Global Expansion:** The EV market is expanding globally, with growth in China, Europe, and other regions. This is driven by government policies, consumer demand, and the availability of local manufacturing.

**Market Outlook**
The EV market is expected to continue its rapid growth in the coming years. According to the International Energy Agency IEA, global EV sales reached 6.6 million in 2021 and are projected to exceed 30 million by 2030. This growth will be driven by the aforementioned trends, including increasing consumer adoption, technological advancements, and government support.

**Conclusion**

The EV market is undergoing a transformative shift. Changes in consumer behavior, technological advancements, and shifts in the competitive landscape are shaping the future of the industry. As EVs become more affordable, practical, and environmentally friendly, they are poised to play an increasingly significant role in the global transportation sector.**Canoo: Financial Performance Analysis**

**Introduction**
Canoo is a high-tech electric vehicle EV company that designs, develops, and manufactures EVs and provides mobility solutions. The company was founded in 2017 and is headquartered in Torrance, California. Canoo's mission is to "bring EVs to Everyone" by offering affordable, sustainable, and innovative vehicles.

**Revenue**
Canoo's revenue has grown significantly in recent years. In 2023, the company reported revenue of $24.3 million, a 43.5% increase from the previous year. This growth was primarily driven by increased sales of the company's Lifestyle Delivery Vehicle LDV.

**Profit Margins**
Canoo's profit margins are still negative, as the company is in the early stages of its development. In 2023, the company reported a gross profit margin of -68.4% and a net loss of $235.9 million. However, the company's gross profit margin has improved significantly in recent quarters, indicating that it is making progress towards profitability.

**Return on Investment ROI**
Canoo's ROI is currently negative, as the company has not yet achieved profitability. However, the company's long-term ROI potential is significant. Canoo has a strong product portfolio, a growing customer base, and a number of strategic partnerships. As the EV market continues to grow, Canoo is well-positioned to generate significant revenue and profits.

**Expense Structure**
Canoo's expense structure is primarily driven by research and development R&D, sales and marketing, and general and administrative G&A expenses. In 2023, the company reported R&D expenses of $92.5 million, sales and marketing expenses of $24.1 million, and G&A expenses of $119.3 million. Canoo's R&D expenses are expected to remain high in the near term, as the company continues to invest in the development of new products and technologies. However, the company's sales and marketing and G&A expenses are expected to decline as a percentage of revenue as the company scales up its operations.

**Financial Outlook**
Canoo expects to continue to grow its revenue and improve its profit margins in the coming years. The company is targeting revenue of $1.7 billion in 2024 and $5.0 billion in 2025. Canoo also expects to achieve profitability in 2024.

**Key Risks**
Canoo faces a number of risks, including:

* **Competition:** The EV market is becoming increasingly competitive, with a number of established automakers and startups entering the market. Canoo will need to differentiate its products and services in order to compete successfully.
* **Technology:** The EV market is rapidly evolving, with new technologies emerging all the time. Canoo will need to stay abreast of the latest technologies in order to remain competitive.
* **Supply Chain:** The EV supply chain is complex and global. Canoo will need to manage its supply chain effectively in order to avoid disruptions.
* **Regulatory:** The EV market is subject to a number of regulations. Canoo will need to comply with these regulations in order to operate its business.

**Conclusion**
Canoo is a high-tech EV company with a strong product portfolio, a growing customer base, and a number of strategic partnerships. The company is well-positioned to benefit from the growing EV market. However, Canoo faces a number of risks, including competition, technology, supply chain, and regulatory risks.

**Financial Performance**

| Metric               | 2022           | 2023           |
|----------------------|----------------|----------------|
| Revenue              | $17.0 million  | $24.3 million  |
| Gross profit         | -$38.0 million | -$16.7 million |
| Gross margin         | -222.4%        | -68.4%         |
| Net income (loss)    | -$305.7 million | -$235.9 million |
| Net margin           | -1,798.2%      | -970.8%        |
| EBITDA               | -$49.2 million | -$37.1 million |
| Adjusted EBITDA      | -$38.0 million | -$16.7 million |
| Total assets         | $369.9 million | $455.6 million |
| Total liabilities    | $308.6 million | $335.1 million |
| Shareholders' equity | $61.3 million  | $120.5 million |

**Expense Structure**

| Category                | 2022           | 2023           |
|-------------------------|----------------|----------------|
| Research and development | $92.5 million  | $73.5 million  |
| Sales and marketing     | $24.1 million  | $30.7 million  |
| General and administrative | $119.3 million | $131.7 million |

**Key Financials**

| Metric               | Value         |
|----------------------|---------------|
| Market capitalization | $1.2 billion |
| Enterprise value     | $1.1 billion |
| Price-to-sales ratio | 49.5x         |
| Price-to-book ratio  | 9.9x          |
| EV/EBITDA ratio     | -23.6x         |
| EV/Sales ratio       | 45.1x         |


**Disclaimer**
The information contained in this report is for informational purposes only and should not be construed as investment advice. Please consult with a financial advisor before making any investment decisions.